# Did you know you can add control functions to Generators?

Generators support two methods:

`send()` and `throw()`

In [14]:
def my_multiplier_generator(multiply_by, run_while_values_bigger_than):
    received = yield
    while received < run_while_values_bigger_than:
        print(f"received = {received}")
        received = yield received * multiply_by
    
    yield received * multiply_by
        

it = iter(my_multiplier_generator(2, 10))

for i in [None, 3,2,1,6,8,11]:
    try:
        output = it.send(i)
        print(f"output: {output}")
    except StopIteration:
        pass

output: None
received = 3
output: 6
received = 2
output: 4
received = 1
output: 2
received = 6
output: 12
received = 8
output: 16
output: 22


In [16]:
class MyError(Exception):
    pass

def my_generator():
    yield 1
    yield 2
    yield 3

it = my_generator()
print(next(it))
print(next(it))
print(it.throw(MyError('test error')))

1
2


MyError: test error

In [19]:
def my_controlled_generator():
    yield 1
    
    try:
        yield 2
    except MyError:
        print('Got my error')
    else:
        yield 3
    
    yield 4

it = my_controlled_generator()
print(next(it))
print(next(it))
print(it.throw(MyError("test error")))

1
2
Got my error
4


In [22]:
class Reset(Exception):
    pass

def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

def announce(remaining):
    print(f'{remaining} ticks remaining')

def check_for_reset_it(): # Replicate some external system check
    yield False
    yield False
    yield False
    yield True # reset here
    while True: yield False

check_for_reset = iter(check_for_reset_it())

def run():
    it = timer(4)
    while True:
        try:
            if next(check_for_reset):
                current = it.throw(Reset())
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)
run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining


## Code works! BUT is hard to read and very odd

It is not recommended to use either the `send()` or `throw()` methods of a generator. Use a defensive generator container with the desired functionality

In [24]:
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period
    
    def reset(self):
        self.current = self.period
    
    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current
            
            
check_for_reset = iter(check_for_reset_it())

def run():
    timer = Timer(4)
    for current in timer:
        if next(check_for_reset):
            timer.reset()
        announce(current)
run()

3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining
3 ticks remaining
2 ticks remaining
1 ticks remaining
0 ticks remaining
